In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

2025-05-16 14:09:00.557027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747404540.619371     110 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747404540.636673     110 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model_name = "vennify/t5-base-grammar-correction"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
def correct_grammer(sentence, prefix="Correct the grammer "):
    input_text = f"{prefix}: {sentence}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_length=100)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if decoded.lower().startswith(prefix.lower()):
        decoded = decoded[len(prefix):].lstrip(": ").strip()
    return decoded

In [4]:
import nltk
import random
import spacy
from nltk.corpus import wordnet as wn

nlp = spacy.load("en_core_web_sm")

In [5]:
def get_synonym(word, pos_tag):
    """Return a synonym of the word using WordNet, or the original word if none found."""
    pos_map = {
        'NOUN': wn.NOUN,
        'VERB': wn.VERB,
        'ADJ': wn.ADJ,
        'ADV': wn.ADV
    }
    wn_pos = pos_map.get(pos_tag, None)
    if wn_pos is None:
        return word

    synonyms = wn.synsets(word, pos=wn_pos)
    lemmas = set()
    for syn in synonyms:
        for lemma in syn.lemmas():
            if lemma.name().lower() != word.lower():
                lemmas.add(lemma.name().replace("_", " "))
    if lemmas:
        return random.choice(list(lemmas))
    return word

def synonym_paraphraser(sentence, replacement_rate=0.3):
    """Replace some words in the sentence with synonyms based on replacement_rate."""
    doc = nlp(sentence)
    new_tokens = []

    for token in doc:
        if (
            token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV'] and 
            not token.is_stop and 
            token.is_alpha and 
            random.random() < replacement_rate
        ):
            synonym = get_synonym(token.text, token.pos_)
            new_tokens.append(synonym)
        else:
            new_tokens.append(token.text)

    return " ".join(new_tokens)


In [10]:
from nltk.tokenize import sent_tokenize

In [11]:
def split_text(text):
    return sent_tokenize(text)

In [17]:
def convert_text(text):
    sentences = split_text(text)
    print(len(sentences))
    ans = ""
    for sentence in sentences:
        sentence = synonym_paraphraser(sentence, replacement_rate=0.3)
        sentence = correct_grammer(sentence)
        ans += sentence.strip()
        if not sentence.strip().endswith((".", "!", "?")):
            ans += ". "
        else:
            ans += " "
    return ans.strip()

In [22]:
text = """The methodology for kidney disease detection using deep learning with the VGG16 model is as follows:

Data Collection
A dataset of kidney images was used, comprising of two classes: normal kidneys and those affected by tumors. The images were collected from Kaggle image datasets, which had sufficient representation of both classes to create a complete dataset.

Data Preprocessing
Following steps of preprocessing were used:

Image Resizing: All images were resized to 224x224 pixels to match the input size required by the VGG16 architecture.

Normalization: The pixel values of the images were normalized to a range of 0 to 1 by dividing by 255. This step makes the execution of the model faster as well as accurate.

Model Selection
The VGG16 convolutional neural network (CNN) was selected for the classification of images. VGG16 is a deep learning model which was trained on the Kaggle dataset, which had thousands of kidney CT scan images. To level up the pre-trained weights, the VGG16 model was connected with the top fully connected layers removed. This improved the model’s capability.

Model Architecture
The architecture of the model is as follows:

Primary Model: The convolutional layers of VGG16 model were used as the base for feature extraction. These layers were frozen in order to prevent any loss in pre-trained weights.

Mean Pooling: A mean pooling layer was used after the convolutional layer to reduce shape of the maps.

Fully Connected Layer: A series of fully connected layers were applied to the model, with the final layer using a softmax activation function for binary classification between normal and tumor classes.

Model Training
The VGG16 model was trained with a softmax activation function, that had a learning rate of 0.001 to guide the training process. The dataset was divided into two sections, with 80% used for training and the remaining 20% reserved for testing. Training was carried out over 40 epochs to enhance the model’s performance and efficiency.

Evaluation
The model’s performance was assessed using the validation dataset. To evaluate its classification capability, standard metrics including accuracy, precision, and loss were employed."""

In [23]:
print(convert_text(text))

The methodological analysis for kidney disease detection using deep learning with the VGG16 example is as keep up : Data Collection A dataset of kidney images was used, comprising of two families : normal kidneys and those affected by tumors. The images were collected from Kaggle image datasets , which had sufficient representation of both classes to make an array dataset. Data Preprocessing Following steps of preprocessing were used : Image Resizing : All images were resized to 224x224 pixels to couple the input sizing required by the VGG16 computer architecture . Correct the grammar: standardisation: The pixel values of the images were normalized to a range of 0 to 1 by dividing by 255 . This footstep makes the writ of execution of the framework faster as well as accurate. Model Selection The VGG16 convolutional neural net ( CNN ) was selected for the compartmentalization of images. VGG16 is a thick erudition model which was trained on the Kaggle dataset , which had thousands of kidn